### 🏆 최종 실습 과제: 실제 도서 리뷰 데이터로 숨은 목소리 찾기

지금까지 배운 토픽 모델링과 군집화 기법을 실제 데이터에 적용하여 독자들이 남긴 리뷰 속에 숨겨진 다양한 목소리와 주제를 발견해 봅시다.

**과제 목표:**
교보문고의 특정 베스트셀러 도서에 대한 리뷰를 직접 수집(크롤링)하고, 비지도 학습을 통해 리뷰들을 주제별로 묶고(토픽 모델링), 비슷한 내용의 리뷰들을 그룹화(군집화)하여 인사이트를 도출합니다.

#### 📚 1단계: 데이터 수집 (웹 크롤링)

먼저 분석할 리뷰 데이터를 수집해야 합니다. 제가 알려드리는 방식으로 수집을 해보세요(별도 교육)

* **대상 도서:** 트렌드 코리아 2025
* **대상 URL:** `https://product.kyobobook.co.kr/detail/S000214208202`
* **수집 내용:** 리뷰 텍스트

In [1]:
"""
fetch("https://product.kyobobook.co.kr/api/review/list?page=1&pageLimit=10&reviewSort=001&revwPatrCode=000&saleCmdtid=S000214208202", {
  "headers": {
    "accept": "*/*",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Google Chrome\";v=\"137\", \"Chromium\";v=\"137\", \"Not/A)Brand\";v=\"24\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin"
  },
  "referrer": "https://product.kyobobook.co.kr/detail/S000214208202",
  "referrerPolicy": "strict-origin-when-cross-origin",
  "body": null,
  "method": "GET",
  "mode": "cors",
  "credentials": "include"
});
"""

# 교보문고 API를 통한 리뷰 데이터 수집
import requests
import pandas as pd
import json
import time

# 리뷰 데이터를 저장할 리스트
all_reviews = []

# 여러 페이지에서 리뷰 수집 (1~10페이지)
for page in range(1, 11):
    url = f"https://product.kyobobook.co.kr/api/review/list?page=1&pageLimit=10&reviewSort=001&revwPatrCode=000&saleCmdtid=S000214208202"
    
    headers = {
        "accept": "*/*",
        "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "cache-control": "no-cache",
        "pragma": "no-cache",
        "priority": "u=1, i",
        "sec-ch-ua": "\"Google Chrome\";v=\"137\", \"Chromium\";v=\"137\", \"Not/A)Brand\";v=\"24\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"macOS\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            
            # 리뷰 데이터 추출 - API 응답 구조에 맞게 수정
            if 'data' in data and 'reviewList' in data['data']:
                reviews = data['data']['reviewList']
                for review in reviews:
                    review_text = review.get('revwCntt', '').strip()
                    if review_text:  # 빈 리뷰가 아닌 경우만 추가
                        all_reviews.append({
                            'review_text': review_text,
                            'rating': review.get('revwRvgr', 0),
                            'date': review.get('cretDttm', '')
                        })
            
            print(f"페이지 {page} 수집 완료 - 현재까지 {len(all_reviews)}개 리뷰")
            
        else:
            print(f"페이지 {page} 요청 실패: {response.status_code}")
            
    except Exception as e:
        print(f"페이지 {page} 수집 중 오류 발생: {e}")
    
    # 서버 부하 방지를 위한 대기
    time.sleep(1)

# DataFrame으로 변환
review_df = pd.DataFrame(all_reviews)
print(f"\n총 {len(review_df)}개의 리뷰를 수집했습니다.")
print(review_df.head())

# 빈 리뷰 제거
review_df = review_df[review_df['review_text'].str.strip() != '']
print(f"\n전처리 후 {len(review_df)}개의 리뷰가 남았습니다.")

페이지 1 수집 완료 - 현재까지 10개 리뷰
페이지 2 수집 완료 - 현재까지 20개 리뷰
페이지 3 수집 완료 - 현재까지 30개 리뷰
페이지 4 수집 완료 - 현재까지 40개 리뷰
페이지 5 수집 완료 - 현재까지 50개 리뷰
페이지 6 수집 완료 - 현재까지 60개 리뷰
페이지 7 수집 완료 - 현재까지 70개 리뷰
페이지 8 수집 완료 - 현재까지 80개 리뷰
페이지 9 수집 완료 - 현재까지 90개 리뷰
페이지 10 수집 완료 - 현재까지 100개 리뷰

총 100개의 리뷰를 수집했습니다.
                                         review_text  rating  \
0                          이 책이 나오면 겨울이 온다라고 생각이 듭니다       4   
1  매년 나오는데  24년트렌드는 구매하고 읽지도 못하고 한해가 거의 지나가네요 \n2...       3   
2           10분만 투자하면 전체 내용을 파악할 수 있는, 가벼이 훑어보면 되는 책       2   
3                                  매년 운세 보듯이 보게 되는 책       4   
4  소비자 트렌드 관점에서는 새로운 이슈 출현을 강조 하고자 했으나 점점 트렌드를 억측...       2   

                         date  
0  2024-09-05 09:39:31.523351  
1  2024-09-26 18:15:42.989277  
2  2024-09-30 14:43:19.030848  
3  2024-09-25 12:10:49.673595  
4  2024-10-14 06:33:38.571394  

전처리 후 100개의 리뷰가 남았습니다.



#### 🔍 2단계: 토픽 모델링 (LDA)으로 리뷰 주제 파악하기

수집한 리뷰들에는 어떤 숨겨진 주제들이 있을지 LDA를 통해 분석해 봅시다.

1.  **DTM 생성:** 전처리된 'processed' 데이터를 `CountVectorizer`를 사용하여 DTM(단어-문서 행렬)으로 변환하세요.
2.  **LDA 모델 학습:** `LatentDirichletAllocation`을 사용해 **4개의 토픽**을 추출해 보세요.
3.  **결과 해석:**
    * 각 토픽을 대표하는 상위 5~7개의 키워드를 출력하세요.
    * 키워드를 바탕으로 각 토픽에 **이름을 붙여보세요.** 예를 들어, "실천과 변화", "선물 및 추천", "번역 및 가독성" 등과 같이 해석할 수 있습니다. 이를 통해 독자들이 어떤 관점에서 이 책을 평가하는지 파악할 수 있습니다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 1. DTM 생성 (전처리: 불용어 제거, 소문자화 등은 간단히 진행)
import re

def preprocess_text(text):
    # 한글, 영문, 숫자만 남기고 나머지 제거, 소문자화
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", " ", text)
    text = text.lower()
    return text

review_df['processed'] = review_df['review_text'].apply(preprocess_text)

# 불용어 리스트 (간단 예시, 필요시 확장)
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

vectorizer = CountVectorizer(stop_words=stopwords, max_df=0.95, min_df=2)
dtm = vectorizer.fit_transform(review_df['processed'])

# 2. LDA 모델 학습 (토픽 4개)
lda = LatentDirichletAllocation(n_components=4, random_state=42)
lda.fit(dtm)

# 3. 각 토픽별 상위 키워드 출력
def print_top_words(model, feature_names, n_top_words=7):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\n[토픽 {topic_idx+1}]")
        top_features = topic.argsort()[:-n_top_words - 1:-1]
        keywords = [feature_names[i] for i in top_features]
        print("키워드:", ", ".join(keywords))

print("각 토픽별 상위 키워드:")
print_top_words(lda, vectorizer.get_feature_names_out(), n_top_words=7)



#### 🧩 3단계: K-Means 군집화로 유사 리뷰 그룹화하기

비슷한 내용을 담고 있는 리뷰들을 그룹으로 묶어 봅시다.

1.  **TF-IDF 행렬 생성:** 전처리된 'processed' 데이터를 `TfidfVectorizer`를 사용하여 TF-IDF 행렬로 변환하세요.
2.  **K-Means 모델 학습:** `KMeans`를 사용하여 **4개의 군집**으로 리뷰들을 나누세요.
3.  **결과 분석:**
    * 원본 `review_df`에 'cluster\_id' 컬럼을 추가하여 각 리뷰가 어떤 군집에 속하는지 확인하세요.
    * 각 군집별로 리뷰 내용을 몇 개씩 출력하여, 그룹이 어떤 기준으로 묶였는지(e.g., 긍정적 실천 후기, 책의 구성 칭찬, 배송 관련 등) 그 특징을 분석해 보세요.


#### 📊 4단계: 시각화로 군집 결과 확인하기

군집화 결과를 PCA나 t-SNE를 이용해 2차원 공간에 시각화하여 그룹이 잘 형성되었는지 확인합니다.

1.  **차원 축소:** `PCA`나 `t-SNE`를 사용해 TF-IDF 행렬을 2개의 주성분으로 축소하세요.
2.  **산점도 시각화:** `plotly.express`를 사용해 결과를 산점도로 그리세요.
    * 각 점의 색상은 `cluster_id`로 구분합니다.
    * 마우스를 점 위에 올렸을 때 원본 리뷰(`review`)가 표시되도록 설정하여, 각 군집의 분포와 특징을 시각적으로 탐색해 보세요.


#### ✨ 도전 과제

* 토픽과 군집의 개수(`n_components`, `n_clusters`)를 3, 5 등 다른 숫자로 변경하며 결과를 비교해 보세요. 어떤 개수가 가장 해석하기 좋은 결과를 도출하나요?
* 다른 책(예: 소설, 에세이)의 리뷰를 수집하여 동일한 분석을 수행하고, 책의 장르에 따라 리뷰의 주제와 군집이 어떻게 달라지는지 비교 분석해 보세요.